In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

path_to_raw_data = Path("../src/data/raw")
path_to_processed_data = Path("../src/data/processed")
path_to_copilot_table = path_to_raw_data / "delta_change_copilot.csv"

In [2]:
# Read the delta change method values from the GEUS table
df_copilot = pd.read_csv(path_to_copilot_table)
df_copilot = df_copilot[np.logical_and.reduce((df_copilot["region"]=="DK5", df_copilot["scenario"]=="far", df_copilot["metric"]=="dc"))]
df_copilot.drop(["region", "scenario", "metric", "page"], axis=1, inplace=True)
df_copilot["month"] = np.select(
    [
        df_copilot["month"]=="Jan",
        df_copilot["month"]=="Feb",
        df_copilot["month"]=="Mar",
        df_copilot["month"]=="Apr",
        df_copilot["month"]=="May",
        df_copilot["month"]=="Jun",
        df_copilot["month"]=="Jul",
        df_copilot["month"]=="Aug",
        df_copilot["month"]=="Sep",
        df_copilot["month"]=="Oct",
        df_copilot["month"]=="Nov",
        df_copilot["month"]=="Dec",
    ],
    np.arange(1,13)
)

In [6]:
# Apply delta changes (factor for precipitation/epot and absolute for temperature)
ts = pd.read_csv(path_to_processed_data / "timeseries_corrected.csv")
ts = ts[
    np.logical_and.reduce((
        pd.to_datetime(ts["date"]).dt.year >= 1971, # Not part of reference period
        pd.to_datetime(ts["date"]).dt.year <= 2000, # Not part of reference period
    ))
]
ts["date"] = pd.to_datetime(ts["date"])
ts["month"] = ts["date"].dt.month

future_dates = pd.date_range("2069-01-01", "2098-12-31")

precipitation_future = ts[["date", "month", "p"]].copy()
precipitation_future = precipitation_future.iloc[:-1] # Drop the last day, because it needs to match future period length
precipitation_future["date_future"] = future_dates
precipitation_future = precipitation_future.merge(
    df_copilot[df_copilot["variable"]=="Precipitation"].drop(["variable"], axis=1),
    how="left",
    on="month"
)
precipitation_future["p_future"] = precipitation_future["p"] * precipitation_future["value"]

evapotranspiration_future = ts[["date", "month", "epot"]].copy()
evapotranspiration_future = evapotranspiration_future.iloc[:-1] # Drop the last day, because it needs to match future period length
evapotranspiration_future["date_future"] = future_dates
evapotranspiration_future = evapotranspiration_future.merge(
    df_copilot[df_copilot["variable"]=="Reference ET"].drop(["variable"], axis=1),
    how="left",
    on="month"
)
evapotranspiration_future["epot_future"] = evapotranspiration_future["epot"] * evapotranspiration_future["value"]

temperature_future = ts[["date", "month", "t"]].copy()
temperature_future = temperature_future.iloc[:-1] # Drop the last day, because it needs to match future period length
temperature_future["date_future"] = future_dates
temperature_future = temperature_future.merge(
    df_copilot[df_copilot["variable"]=="Temperature"].drop(["variable"], axis=1),
    how="left",
    on="month"
)
temperature_future["t_future"] = temperature_future["t"] + temperature_future["value"]

# Merge results, save to csv
future = precipitation_future[["date_future", "model", "p_future"]].merge(
    temperature_future[["date_future", "model", "t_future"]].merge(
        evapotranspiration_future[["date_future", "model", "epot_future"]],
        how="inner",
        on=["date_future", "model"]
    ),
    how="inner",
    on=["date_future", "model"]
).rename({
    "date_future": "date",
    "p_future": "p",
    "t_future": "t",
    "epot_future": "epot",
}, axis=1)
future.to_csv(path_to_processed_data / "timeseries_future_deltachange.csv", index=False)

In [8]:
def ingest_model(df, model):
    df = df.copy()
    df["model"] = model
    return df

precip_dbs = pd.concat([ingest_model(df,k) for k,df in pd.read_excel(path_to_raw_data / "Precip_DBS.xlsx", sheet_name=None).items()])
et_dbs = pd.concat([ingest_model(df,k) for k,df in pd.read_excel(path_to_raw_data / "RefET_DBS.xlsx", sheet_name=None).items()])
temp_dbs = pd.concat([ingest_model(df,k) for k,df in pd.read_excel(path_to_raw_data / "Temp_DBS.xlsx", sheet_name=None).items()])

dbs = precip_dbs.merge(
    et_dbs.merge(
        temp_dbs,
        on=["Time", "model"],
        how="inner"
    ),
    on=["Time", "model"],
    how="inner"
)
dbs.rename({
    "Time": "date",
    "Precip (mm)": "p",
    "RefET (mm)": "epot",
    "Temp (°C)": "t",
}, axis=1, inplace=True)
dbs["date"] = pd.to_datetime(dbs["date"])

dbs = dbs[np.logical_and(
    dbs["date"].dt.year >= 2069,
    dbs["date"].dt.year <= 2098,
)]

dbs.to_csv(path_to_processed_data / "timeseries_future_dbs.csv", index=False)